In [1]:
#Mounting the Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/projects/chat-bot'
#Please upload the files in your drive and change the path to it accordingly.

Mounted at /content/drive


In [3]:
#2 Importing Relevant Libraries
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
#Loading the Dataset: intents.json
data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

In [7]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Are you cash only?'],
   'responses': ['We accept VISA, Mastercard and AMEX',
    'We accept most major credit c

In [8]:
#Creating data X and data_Y

words = [] #For Bow nodel/ vocabulary for patterns
classes = [] #For Bow model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data X

# Iterating over all the intents

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern) # tokenize each pattern.
    words.extend(tokens) #and append tokens to words
    data_X.append(pattern) #appending pattern to data X
    data_y.append(intent["tag"]), #appending the associated tag to each pattern

  # adding the tag to the classes if it's not there already
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()


#lemmatize all the words in the vocab and convert them lowercase
#if the words dont apeear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
#sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [9]:
#5 Text to Numbers
training = []
out_empty = [0]*len(classes)
# creating the bag of words model

for idx, doc in enumerate(data_X):
  bow  =[]
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  # mark the index of class that the current pattern is associated
  #to
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1
  #add the one hot encoded Boll and associated classes to training
  training.append([bow, output_row])
#shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
#split the features and target labels
train_X= np.array(list(training[:, 0]))
train_Y= np.array(list(training[:, 1]))

In [13]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense (64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               4992      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 8)                 520       
                                                                 
Total params: 13768 (53.78 KB)
Trainable params: 13768 (53.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
1

In [21]:
#7 Preprocessing the Input
def clean_text(text):
  tokens = nltk.word_tokenize (text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words (text, vocab):
  tokens = clean_text(text)
  bow = [0]*len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx]=1
  return np.array (bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
     return_list.append(labels[r[0]]) #Contains labels (tags) for highest probability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [22]:
#8 Interacting with the chatbot
print("Press Ø if you don't want to chat with our ChatBot.")
while True:
  message = input("")
  if message == "0":
    break
  intents=pred_class(message, words, classes)
  result = get_response (intents, data)
  print(result)

Press Ø if you don't want to chat with our ChatBot.
hi
1/1 [==============================] - 0s 23ms/step
Hi there, how can I help?
how are you
1/1 [==============================] - 0s 22ms/step
Hi there, how can I help?
good morning
1/1 [==============================] - 0s 30ms/step
Good to see you again
how do you do
1/1 [==============================] - 0s 21ms/step
Hi there, how can I help?
hello
1/1 [==============================] - 0s 26ms/step
Hi there, how can I help?
what is you name
1/1 [==============================] - 0s 25ms/step
weather today : 30
what is alpha
1/1 [==============================] - 0s 22ms/step
weather today : 30
then
1/1 [==============================] - 0s 22ms/step
Hi there, how can I help?
i am goo
1/1 [==============================] - 0s 117ms/step
Hello, thanks for visiting
can i talk to you
1/1 [==============================] - 0s 23ms/step
My pleasure
my name is arun
1/1 [==============================] - 0s 22ms/step
Hi there, how can I